# django与html 数据传输中的编码解码问题

## 一、遇到的问题
    1.html回传给django的参数如果包含中文，在django中获取到是乱码。在pycharm中测试可能没有问题，但是打包后会遇到各种问题。
    2.django传送给html的参数，如果通过js接收，莫名奇妙会变成变量名，还会报错undefined。
    3.html接收参数后，回传给django，中文部分会乱码。

## 二、编码与解码的方法

### 1.django部分

In [10]:
from urllib import parse #这个库用来编码解码的。

#编码
txt="?..你_-好"
bianma=parse.quote(txt)
print("编码 :%s"%bianma)

#安全转码，只转码中文字符
import string
bianma2=parse.quote(txt,safe=string.printable)
print("编码2:%s"%bianma2)

编码 :%3F..%E4%BD%A0_-%E5%A5%BD
编码2:?..%E4%BD%A0_-%E5%A5%BD


In [8]:
#解码
jiema=parse.unquote(bianma)
print("解码:%s"%jiema)

解码:你_-好


## 2.html部分
    html本身是静态语言，直接显示传过来的参数不会有影响。
    关键是接收到的参数通过url形式再反传给django的参数如果带有中文，会显示乱码。示例如下

**上面的方法将参数回传“127.0.0.1/download/”的页面，django接收到参数后，如果里面带有中文，就会显示乱码**

## 3.js部分

# 三、html与django参数的接收与发送
## 1.dango传递参数

## 2.html部分
### （1）使用方法
    使用以下方法遍历django传递过来的参数。即上面的两个参数[{'path':resultPath,'name':fileName}]
    {% for “value值” in “key值” %}......{% endfor %}

    在省略号中使用{{ “value值”.“参数名” }}获取参数信息
### （2）注意
    通过js获取参数后，貌似参数值变成了变量名，解码的时候会报错。而使用html通过上面的方法不受影响（因为本身就是静态语言）。

# 3.js部分（重要）

js如果要接收参数需要加引号，如var a="{{ i.name }}"；否则莫名其妙会变成变量。

In [10]:
<div class="result-style">
        {% for i in result %}
            <table class="table-style">
                <script language="javascript">
                    var get_name={{ i.name }}
                    var name=unescape(get_name.toString())

                    if (name == "" || name == null || name == undefined){
                        document.getElementById('show_result').innerHTML=name
                    }else {
                        document.getElementById('show_result').innerHTML="oo"
                    }
                    console.log('name is',name)

                </script>
                
                <tr >
                    <a href='download/?path={{ i.path }}&name={{ i.name }}' id="show_result">无搜索结果</a>
                </tr>
            </table>
        {% endfor %}
    </div>

SyntaxError: invalid syntax (<ipython-input-10-a0fba58fd8d4>, line 1)

# 四、解决方案
## 1.解决思路（没有印象，后续完善）
    先通过django进行编码（将编码的参数 和 不编码的参数分成两个变量都传递过去，这样方面html处理）
        ↓
    html接收参数后不进行解码，直接显示。
    js接收传输，必须要加引号。再使用unescape进行解码。（这一步貌似遇到问题，记不清了，后面遇到问题再修改）
        ↓
    html发送把参数回传时，使用编码过的参数